# Gradient Descent
With Wolfe conditions on step size.

In [270]:
import numpy as np
import scipy.optimize
from numpy.linalg import norm

def gradient_descent(fun, x0, grad, args=(), eps=0.01, c2=0.05, max_iter=10):
    alpha = 0.3
    beta = 0.8
    x = x0
    f = fun(x0, *args)
    g = grad(x, *args)
    f_evals, g_evals = 1, 1
    i = 0
    print(i, f)
    while norm(g) > eps and i < max_iter:
        step, _, _, _, _, _ = scipy.optimize.line_search(
            fun, grad, x, -g, args=args, old_fval=f)
        method = "linesearch"
        if step is None:
            method = "backtrack"
            step = backtracking_line_search(fun, x, grad, -g, alpha, beta, args)
        x -= step * g
        f = fun(x, *args)
        g = grad(x, *args)
        i += 1
        print(i, method, step, f, norm(g))
        #f_evals += fc
        #g_evals += (gc + 1)
    return x#, f_evals, g_evals


# Line search function returns optimal step size.
def backtracking_line_search(fun, x, grad, delta_x, alpha, beta, args=()):
    t = 1
    derprod = grad(x, *args) @ delta_x
    while fun((x + (t * delta_x)), *args) > fun(x, *args) + (alpha * t * derprod):
        t *= beta
    return t

## Least-squares problem

In [262]:
np.random.seed(0)
m, n = 10, 5
a = np.random.random((m, n))
b = np.random.random((m, ))
x0 = np.random.random((n, ))

def f(x):
    return 0.5 * norm(a @ x - b) ** 2

def grad(x):
    return a.T @ (a @ x - b)

print(gradient_descent(f, x0, grad, max_iter=20))
print(scipy.optimize.fmin(f, x0))

0 1.0418609054066488
1 linesearch 0.06675468176963371 0.46741882064763834 0.7793256749695983
2 linesearch 0.4975785125302646 0.31631703713147225 1.0943220788762398
3 linesearch 0.06712095468338468 0.27612699583398875 0.22213092223376688
4 linesearch 1.0 0.2716354685252032 0.7046350503237157
5 linesearch 0.06576179311753909 0.2553097563540339 0.10448496105038993
6 linesearch 1.0 0.25176311442909644 0.24494409134157516
7 linesearch 0.06743577594633629 0.24974012180783442 0.052991893359859564
8 linesearch 1.0 0.24911451016478864 0.1748511477236272
9 linesearch 0.06593099130741929 0.24810665857615172 0.02765353332188246
10 linesearch 1.0 0.24774819622865044 0.06762696670336901
11 linesearch 0.06773470753160053 0.24759330704854982 0.015450192456231963
12 linesearch 1.0 0.24751606720760017 0.05274521924794974
13 linesearch 0.06604641370062372 0.24742419472572713 0.008681318690592096
[ 0.007395    0.53439085  0.35932505  0.03808262 -0.27715026]
Optimization terminated successfully.
         C

## Least-Squares of Quadratic Form

In [300]:
np.random.seed(0)

m, n = 80, 80
a = [np.random.random((n, n)) for _ in range(m)]
b = np.random.random((m, ))
x0 = np.random.random((n, ))

def f(x):
    return sum((x.T @ ai @ x - bi) ** 2 for ai, bi in zip(a, b))

def grad(x):
    return 2 * sum((x.T @ ai @ x - bi) * (ai + ai.T) @ x for ai, bi in zip(a, b))

In [ ]:
# # Gradient check.
# np.random.seed(0)
# x = np.random.random((n,))

# def unit_vector(n, i):
#     e = np.zeros((n,))
#     e[i] = 1
#     return e

# delta = 1e-7
# grad_fd = np.array([(f(x + delta * unit_vector(n, i)) - f(x)) / delta for i in range(n)])
# print(norm(grad_fd - grad(x)) / norm(grad(x)))

In [303]:
%time x = scipy.optimize.fmin(f, x0)
print(x)
#print(gradient_descent(f, x0, grad, max_iter=100))

CPU times: user 6.64 s, sys: 42.8 ms, total: 6.68 s
Wall time: 6.69 s
[ 0.65879983  1.21253421  0.54111582 -0.46109869  0.19408056 -0.07164759
 -0.67479116 -2.22961714 -0.25741955 -0.7795725   0.10484649  0.64737748
 -0.92311535  0.73118469 -0.15743764 -0.40015546  0.70317633 -0.78449119
 -1.44163058 -1.44965145  1.74575699 -0.21061952 -0.07639984 -0.21563412
  1.59295473  0.0820909   0.56320606 -0.0579347   0.03973153  0.03320395
  0.22529907  0.06793408  0.36024279 -0.14452085 -1.41856998  0.76097638
 -0.48060303 -0.94057674 -0.07303802  0.41632912 -1.46558892  0.14705121
  0.42442239  0.39442216  1.03093412 -0.41449112  1.08782641 -0.02209701
  0.12366279  0.23438518 -0.06018055 -0.21780664  0.84946902 -1.00232965
  0.01703008  1.12505547 -0.81597275  1.01204344  0.21852272 -1.43823989
 -0.87057594  0.52859765  0.53800597 -1.30881082  1.11275872  0.40696492
  0.98363623 -0.83002711 -0.5305062   0.29366004  0.41485163 -1.20253342
  0.2139546  -0.39385226  1.3135983   0.79305139  1.47

Gradient descent is reasonably fast but slows down. Generic minimization is slow. But BFGS is faster.

In [305]:
%time x = scipy.optimize.fmin_l_bfgs_b(f, x0, grad)
#print(x)
print(x[1], x[2]['funcalls'])

CPU times: user 1.92 s, sys: 55.5 ms, total: 1.97 s
Wall time: 1.97 s
1.8928110554893907e-06 928


In [ ]:
# # Hessian not worth calculating, too slow (O(n^4) = O(B^8) elements!).
# 78 ** 4
# n = 78
# a = np.random.random((n, n))
# b = np.random.random((n, ))
# %timeit a @ b
# n = 78
# a = np.random.random((n, n, n, n))
# b = np.random.random((n, ))
# %timeit np.tensordot(a, b, axes=1)


# Hessian Check

In [325]:
def unit_vector(n, i):
    e = np.zeros((n,))
    e[i] = 1
    return e

def grad_check(fun, grad, n, delta: float = 1e-7, x = None):
    np.random.seed(0)
    if x is None:
        x = np.random.random((n,))
    f0 = fun(x)
    grad_fd = np.array([(fun(x + delta * unit_vector(n, i)) - f0) / delta for i in range(n)])
    g = grad(x)
    return norm(grad_fd - g) / norm(g)

In [381]:
n = 3
q = np.diag([1, 0, 0]) #np.random.random((n, n))
a = q + q.T

def g(x):
    return (x.T @ q @ x) * (a @ x)

def h(x):
    ax = a @ x
    return (x.T @ q @ x) * a + np.outer(ax, ax)

In [382]:
x = np.ones((n, ))
grad_check(g, h, n, x=x)

1.0050393939309288e-07

In [370]:
delta = 1e-4
h(x)

array([[6., 4., 4.],
       [4., 4., 4.],
       [4., 4., 4.]])

In [371]:
np.array([(g(x+delta * unit_vector(n, i)) - g(x))/delta for i in range(n)])

array([[6.00060002, 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [388]:
n = 3

def g(x):
    g = np.zeros_like(x)
    g[:n] -= 2 * (1 / x[:n] - x[:n] / sum(x[:n] ** 2))
    return g

def h(x):
    g = np.zeros((len(x), len(x)))
    s = sum(x[:n] ** 2)
    g[:n, :n] -= 2 * (np.diag(-1 / x[:n] ** 2) - np.eye(n) / s + 2 * np.outer(x[:n], x[:n]) / s ** 2)
    return g

In [389]:
x = np.ones((n, ))
grad_check(g, h, n, x=x)

1.0124955391634866e-07

In [390]:
np.array([(g(x+delta * unit_vector(n, i)) - g(x))/delta for i in range(n)])

array([[ 2.22198521, -0.44443704, -0.44443704],
       [-0.44443704,  2.22198521, -0.44443704],
       [-0.44443704, -0.44443704,  2.22198521]])

In [391]:
h(x)

array([[ 2.22222222, -0.44444444, -0.44444444],
       [-0.44444444,  2.22222222, -0.44444444],
       [-0.44444444, -0.44444444,  2.22222222]])